In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
df_Alagoas = pd.read_csv('microdadosEnemAlagoas.csv', sep = ';', encoding='ISO 8859-1')

In [9]:
df_Alagoas = df_Alagoas.loc[:, ~df_Alagoas.columns.str.contains('^Unnamed')]

y = df_Alagoas
X = df_Alagoas


In [10]:
df_Alagoas.sample(5)

,NU_INSCRICAO,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,NU_NOTA_CN,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
51211,180012192598,AL,33.0,1.0,0.0,3.0,12.0,1.0,0.0,602.1,...,A,B,B,A,D,A,B,B,A,A
24431,180009779615,AL,16.0,0.0,0.0,3.0,0.0,1.0,1.0,424.3,...,A,B,B,A,E,A,B,B,C,A
52830,180012028277,AL,18.0,0.0,0.0,1.0,2.0,1.0,0.0,493.0,...,A,B,A,A,C,A,A,B,A,D
28598,180009887933,AL,19.0,0.0,0.0,3.0,0.0,2.0,0.0,461.5,...,A,B,A,A,B,A,A,B,B,A
29548,180010121758,AL,34.0,0.0,0.0,1.0,12.0,1.0,0.0,445.7,...,A,D,B,B,E,A,B,B,A,D


In [11]:
# Leitura dos datasets de treino e teste e criação do df de resposta
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)
#df_resposta = pd.DataFrame()
print(X.shape, y.shape, X_train.shape, y_train.shape ,X_test.shape, y_test.shape)

(64605, 41) (64605, 41) (51684, 41) (51684, 41) (12921, 41) (12921, 41)


# Variável alvo é a nota de matematica dos participantes

In [12]:
#dataframe que irá receber a resposta dada pelo sistema
df_resposta = pd.DataFrame()

#dataframe que guarda a nota orginal obtida pelo participante
respOriginal_MT = pd.DataFrame()

df_resposta['NU_INSCRICAO'] = X_test['NU_INSCRICAO']

respOriginal_MT =  X_test[ ['NU_INSCRICAO','NU_NOTA_MT']]
print(df_resposta.sample(5) )
print(respOriginal_MT.sample(5) )
y = X_train['NU_NOTA_MT']


       NU_INSCRICAO
25330  180009636210
20532  180008501092
28573  180009425631
60454  180013804549
16861  180008652057
       NU_INSCRICAO  NU_NOTA_MT
40249  180010736608       452.1
30100  180009842231       465.4
34513  180010577183       540.0
5110   180008894814       515.3
1640   180008437535       472.4


In [19]:
df_Alagoas.columns

Index(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
       'IN_TREINEIRO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015',
       'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024',
       'Q025', 'Q026', 'Q027'],
      dtype='object')

In [20]:
numeric_features = ['NU_IDADE', 'TP_SEXO','TP_ESTADO_CIVIL', 'TP_COR_RACA','TP_ANO_CONCLUIU', 'TP_ESCOLA',
                    'IN_TREINEIRO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO',  'Q005']

numeric_transformer = Pipeline(   steps=[  ('scaler', StandardScaler())])

categorical_features = ['Q001', 'Q002', 'Q003', 'Q004','Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011',
                        'Q012', 'Q013', 'Q014', 'Q015','Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021',
                        'Q022', 'Q023', 'Q024', 'Q025', 'Q026', 'Q027']
categorical_transformer = Pipeline(steps=[   ('onehot', OneHotEncoder(handle_unknown='ignore'))])


X_train = X_train[numeric_features + categorical_features]
X_test = X_test[numeric_features + categorical_features]





preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [21]:
print(X_train, X_test)

       NU_IDADE  TP_SEXO  TP_ESTADO_CIVIL  TP_COR_RACA  TP_ANO_CONCLUIU  \
661        39.0      1.0              1.0          1.0             12.0   
26028      35.0      0.0              1.0          3.0             12.0   
47760      48.0      1.0              0.0          3.0             12.0   
31187      56.0      0.0              0.0          1.0             12.0   
41083      17.0      0.0              0.0          2.0              0.0   
11969      29.0      0.0              1.0          1.0             11.0   
9209       20.0      0.0              0.0          3.0              2.0   
58464      18.0      1.0              0.0          2.0              1.0   
51222      18.0      1.0              0.0          3.0              0.0   
18337      19.0      1.0              0.0          3.0              0.0   
18897      28.0      0.0              0.0          3.0              8.0   
435        21.0      0.0              0.0          3.0              2.0   
20300      18.0      1.0 

In [22]:

# execute the data preparation pipeline
    #train_prepared = full_pipeline.fit_transform(train)
    #test_prepared = full_pipeline.fit_transform(test)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
rf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, n_jobs=-1, warm_start=True, random_state=42))
])

#full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
 #                     ('classifier', pipelineRF )])

paramsRF = { 'regressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'regressor__max_depth': [None, 5, 3, 1]}


#clf = GridSearchCV(rf, hyperparameters, cv=10)
clf = GridSearchCV(rf, paramsRF, cv=10)

In [ ]:
rf_detector = clf.fit(X_train , y)

In [32]:
#print(preprocessor)
print(X_train.head, X_test.sample(5) )
pred_notas = clf.predict(X_test)
print(pred_notas)
print("model score: %.3f" % clf.score(X_train, y))

<bound method NDFrame.head of        NU_IDADE  TP_SEXO  TP_ESTADO_CIVIL  TP_COR_RACA  TP_ANO_CONCLUIU  \
661        39.0      0.0              1.0          1.0             12.0   
26028      35.0      1.0              1.0          3.0             12.0   
47760      48.0      0.0              0.0          3.0             12.0   
31187      56.0      1.0              0.0          1.0             12.0   
41083      17.0      1.0              0.0          2.0              0.0   
11969      29.0      1.0              1.0          1.0             11.0   
9209       20.0      1.0              0.0          3.0              2.0   
58464      18.0      0.0              0.0          2.0              1.0   
51222      18.0      0.0              0.0          3.0              0.0   
18337      19.0      0.0              0.0          3.0              0.0   
18897      28.0      1.0              0.0          3.0              8.0   
435        21.0      1.0              0.0          3.0              2.

[561.4115 500.7305 477.989  ... 487.0605 488.119  502.2975]
model score: 0.924


In [26]:
df_resposta['NU_NOTA_MT'] = np.around(pred_notas,2)

In [28]:
df_resposta.rename(columns={'NU_NOTA_MT': 'NU_NOTA_MT_Preditiva'}, inplace= True)
pd.concat([respOriginal_MT, df_resposta])
print( df_resposta)
df_resposta['NU_NOTA_MT_Original'] =  respOriginal_MT['NU_NOTA_MT']

#print( respOriginal_MT, df_resposta)
respOriginal_MT 
df_resposta

       NU_INSCRICAO  NU_NOTA_MT_Preditiva  NU_NOTA_MT_Original
59340  180013272216                561.41                571.4
10626  180007664987                500.73                587.8
18410  180007324620                477.99                390.9
16294  180008690383                461.09                479.3
59440  180013685090                476.36                480.2
22992  180007851580                553.96                588.3
14099  180007709738                527.68                550.6
7975   180008338306                493.00                390.7
40909  180010784690                545.00                484.5
25569  180009710718                456.57                422.4
8859   180008304276                557.96                506.7
59683  180012755176                465.26                470.5
9441   180008463362                465.85                443.3
16427  180009026378                525.37                685.3
14431  180007702117                488.28              

C:\Users\aleon\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,NU_INSCRICAO,NU_NOTA_MT_Preditiva,NU_NOTA_MT_Original
59340,180013272216,561.41,571.4
10626,180007664987,500.73,587.8
18410,180007324620,477.99,390.9
16294,180008690383,461.09,479.3
59440,180013685090,476.36,480.2
22992,180007851580,553.96,588.3
14099,180007709738,527.68,550.6
7975,180008338306,493.00,390.7
40909,180010784690,545.00,484.5
25569,180009710718,456.57,422.4
